*20 Dec 2021, Julian Mak (whatever with copyright, do what you want with this)

### As part of material for OCES 3301 "Data Analysis in Ocean Sciences" delivered at HKUST

For the latest version of the material, go to the public facing [GitHub](https://github.com/julianmak/academic-notes/tree/master/OCES3301_data_analysis_ocean) page.

In [ ]:
# load some deafult packages

import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import pandas as pd

---------------------------

# 06: statistical tests

Basically more of the same as last time, but less talk and more code, and with different hypothesis tests and data. We will be going through the sea cucumber and the dice example before focusing on the iris example (because my cooked up multivariate data is not very good with doing $F$-tests on). The exercise just gets you to try and do similar things but for the penguin data.

Again, the doing itself is not too bad, but be very careful with your experimental design, choice of test, and the eventual interpretation. The following cheat sheet is probably useful for remember things to be aware of.

## cheat sheet and banana skins to remember

1) you formulate the null hypothesis to be the opposite of what you want to try and show, choose test based on question and other assumptions about the data, choose significance, do test, and based on analysis either REJECT null hypothesis if $p$-value is small or your test statistic is big compared to the threshold, or FAIL TO REJECT null hypothesis otherwise

2) you never try and proof the null hypothesis, you only ever fail to reject it

3) the $p$-value is not the probability whether the null/alternative hypothesis is true or not

4) $p$-value small by itself doesn't mean anything, and it also does not mean result is of practical significance

5) the choice of $\alpha=0.05$ is purely a convention, and the observation is that unlikely event happens almost surely with large samples, so if you run enough tests you will almost surely to get a hit (do not for God sake torture the data until it confesses, this is incredibly bad practice)

6) $p$-value being below the threshold does not mean the null hypothesis is correct, it really just means there is not enough statistical evidence to say anything, nothing more and nothing less

7) $\alpha$ sets your Type I errors of false positives, while $\beta$ relates to the power of your test and is the Typ II error of false negatives

8) the analysis part is just one step, and if anything your experimental design and/or the question you ask is more important, so check that too 

9) you should probably use the confidence interval instead (although we are not touching on it really in this course)

------------------------
# a) (student's) $t$-test

While the Z-test is when we have ample data and (somewhat known) populaton variance $\sigma$ (and technically Gaussian distribution, but CLT probably applies because of large sample size), the **(student's) $t$-test** is designed really with small samples that follow Gaussian distribution in mind, and when we don't know the population variance (notice we can't really rely on CLT here).

> NOTE: "student" is a pseudonym of [William Gosset](https://en.wikipedia.org/wiki/Student%27s_t-test), who was in the employ of Guiness the alcohol company and was publishing under that name for employer-employee contractual obligations.

Just for completeness, in the Z-test the pdf is the Gaussian. In the (standardised) $t$-test it takes the more complicated form

\begin{equation*}
    {\rm pdf} = \frac{\Gamma((\nu + 1)/2}{\sqrt{\pi\nu}\Gamma(\nu/2)}\left(1 + \frac{x^2}{\nu}\right)^{-(\nu+1)/2},
\end{equation*}

where $\Gamma$ is the gamma-function, and $\nu > 0$ is known as the **degree of freedom**, and is related to the sample size. The pdf looks like the below: notice that for small degree of freedom, the pdf has fatter tails, while for large degrees of freedom it tends to the Gaussian pdf (it can be shown above from the formula too).

In [ ]:
df_vec = [1, 2, 3, 5, 10]

ax = plt.axes()
for df in df_vec:
    x = np.linspace(-5, 5, 100)  # standardised
    ax.plot(x, stats.t.pdf(x, df), label=r"$\nu = %i$" % df)
    
ax.plot(x, stats.norm.pdf(x), 'k--', alpha=0.7, label="Gaussian")
ax.set_ylabel("$x$")
ax.set_ylabel("pdf")
ax.legend()
ax.grid()

Below we are essentially redoing the sea-cucumber example as in the previous notebook, but with smaller samples, and testing the means.

## one sample t-test

Here we are testing one sample against the population mean. The null hypothesis is that there is no change. We follow essentially the same recipe, just with a smaller sample size (5 here instead of 100). 

$t$-tests are easier to do in a way because `scipy.stats` already has that built it, and you don't even need to specify the degrees of freedom, because the code already does that for you. The code spits out the $t$-statistic (like the Z-statistic from before) as well as the $p$-value; here we are going to be looking at the $p$-value. We reject the null hypothesis is the $p$-value is smaller than some threshold.

In [ ]:
pop_mean = 3.0

# generate some data for the sake of it
np.random.seed(1)
sample_size, mean, std = 5, pop_mean+0.5, 0.5
data = mean + std * np.random.randn((sample_size))

alpha = 0.05  # standard number from Fisher
t_val, p_val = stats.ttest_1samp(data, pop_mean) # auto-magically takes into account data length to compute
                                                 # relevant t-statistic and the p values accordingly

print("=================")
print("t-test evaluation")
print("=================")
print(f"null hypothesis is that pop mean is {pop_mean}, threshold value of {alpha:.2f}")
print(f"(real sample mean is {mean})")
print(" ")
print(f"two-sided p-value from data sample is {p_val:.7f}")
print()
if p_val < alpha:
    print(f"  p-value smaller than threshold, REJECT null hypothesis")
    print(" ")
    print(f"    *** pop mean != {pop_mean} is likely at the alpha = {alpha} significance ***")
    print(" ")
else:
    print(f"  p-value not smaller than threshold, FAIL TO REJECT null hypothesis")
    print(" ")
    print(f"    *** pop mean = {pop_mean} is not impossible ***")
    print(" ")
    print(f"(consider sample size, threshold etc.)")

> <span style="color:red">**Q.**</span> Interpret the above outcome with the seed set to 1.

> <span style="color:red">**Q.**</span> Consider asking the same questions as last time with the Z-test, such as what happens if you vary the $\alpha$, the differences in means, and sample size. Particularly look at sample size.

## two sample $t$-test

There are two of this, the **relative** case (where the two data sample size needs to be equal), and the **independence** case (where the two data sample size does not need to be equal). This is particularly useful for comparing between a control group and a test group.

Suppose we expose our sea cucumber to diet/exercise/whatever and want to see the effect on weight. The null hypothesis would be the means are the same, and other things are as usual (except here you are using the two sample $t$-test). The code below is for the relative test.

In [ ]:
np.random.seed(2) # try 1 and 2
sample_size, mean, std = 20, 3.0, 0.5
data_before = mean + std * np.random.randn((sample_size))

# artificially going to make her heavier
np.random.seed(100)
data_after = (mean + 0.25) + std * np.random.randn((sample_size))

alpha = 0.05
t_val, p_val = stats.ttest_rel(data_before, data_after)

print("=================")
print("t-test evaluation")
print("=================")
print(f"null hypothesis is that diet has no impact on weight, threshold value of {alpha:.2f}")
print(f"(real answer is it should have: from data, before mean is {np.mean(data_before):.3f}, after mean is {np.mean(data_after):.3f})")
print(" ")
print(f"two-sided p-value from data sample is {p_val:.7f}")
print()
if p_val < alpha:
    print(f"  p-value smaller than threshold, REJECT null hypothesis")
    print(" ")
    print(f"    *** diet has impact on weight, likely at the alpha = {alpha} significance ***")
    print(" ")
else:
    print(f"  p-value not smaller than threshold, FAIL TO REJECT null hypothesis")
    print(" ")
    print(f"    *** diet has no impact is not impossible ***")
    print(" ")
    print(f"(consider sample size, threshold etc.)")
    
# box-and-whisker plot to see the data
fig = plt.figure(figsize=(4, 4))
ax = plt.axes()
ax.boxplot([data_before, data_after])
ax.set_xticklabels(["control", "after"])
ax.set_xlabel(r"sea cucumber")
ax.set_ylabel(r"weight (units)")
ax.grid()

This one is for the independence case, where the two sample sizes are not equal.

In [ ]:
np.random.seed(10)
sample_size, mean, std = 6, 3.0, 0.5
data_before = mean + std * np.random.randn((sample_size))

# not going to do anything to the mean itself
sample_size, mean, std = 8, 3.0, 0.4
data_after = mean + std * np.random.randn((sample_size ))

alpha = 0.05  # standard number from Fisher
t_val, p_val = stats.ttest_ind(data_before, data_after)

print("=================")
print("t-test evaluation")
print("=================")
print(f"null hypothesis is that toy has no impact on weight, threshold value of {alpha:.2f}")
print(f"(real answer here is that it doesn't do anything)")
print(" ")
print(f"two-sided p-value from data sample is {p_val:.7f}")
print()
if p_val < alpha:
    print(f"  p-value smaller than threshold, REJECT null hypothesis")
    print(" ")
    print(f"    *** toy has impact on weight, likely at the alpha = {alpha} significance ***")
    print(" ")
else:
    print(f"  p-value not smaller than threshold, FAIL TO REJECT null hypothesis")
    print(" ")
    print(f"    *** toy has no impact on weight is not impossible ***")
    print(" ")
    print(f"(consider sample size, threshold etc.)")
    
# box-and-whisker plot to see the data
fig = plt.figure(figsize=(4, 4))
ax = plt.axes()
ax.boxplot([data_before, data_after])
ax.set_xticklabels(["control", "after"])
ax.set_xlabel(r"sea cucumber")
ax.set_ylabel(r"weight (units)")
ax.grid()

The quartiles in the box plots are kind of like the confidence intervals: if one of the median lives outside of the other groups inter-quartiles you might expect there to a statistical significance to the difference.

> <span style="color:red">**Q.**</span> Do similar kind of explorations as before for the two sample case. Pay particular attention to the case where you vary the sample size (you have to be really careful of this when you are using the $t$-test).

----------------
# b) $\chi^2$ test

In [ ]:
# chi2 test 
#
# used for categorical data, with the null hypothesis that the two distributions are the same
# lets take an expanded version of the heads_or_tails example from 05 and turn this into dice
#

# Q. (harder) explain what the subroutine below is doing

def throw_n_sided_dice(side=6, bias=0):
    """
        use a brute force method to mimic dice throw 
        (could just use randint for a fair dice, might be a better way to do loaded dice)
    """
    a = np.random.rand()  # calls from a uniform distribution
    p = 1/side
    for i in range(side):
        if (a > i*p) & (a < (i+1)*p):     # if it is inside some interval, output
            if (np.random.rand() < bias): # dirty hack to bias the dice LOW (if bias = 0 then no bias)
                return np.max([1, i - np.random.randint(1, side)])
            else:
                return i+1

side = 6                         # choose number of sides a dice has
n = 120                          # throw n times
rolls = np.zeros(n, dtype=int)   # force output to be integers
for i in range(n):
    rolls[i] = throw_n_sided_dice(side=side)
print(f"actual output of rolls = {rolls}")

freq = np.zeros(side)
for i in range(side):
    freq[i] = np.sum(rolls == i+1)          # Q. why the +1s etc?
roll_val = np.arange(side) + 1
expected_freq = 1/side * np.ones(side) * n  # Q. what is this doing?

# write this out into a pandas data frame for ease of reading
print()
print(f"=================================================")
print(f"summary table for the statistics of throw of dice")
print(f"=================================================")
df = pd.DataFrame(columns = ["roll_val", "freq", "expected_freq"])
df["roll_val"], df["freq"], df["expected_freq"] = roll_val, freq, expected_freq
print(df)

In [ ]:
# data here counts as categorical data (even if it the category is numerical here) because you can't roll a 1.5
# freq is the summary of an actual experiment, while the expected_freq is what one might expect of a fair dice
# so you could do hypothesis testing to ask "is this dice fair?" (at least from a statistics point of view)
#
# so it is the usual approach as in the other tests
# 1) null hypothesis: the frequency distributions are the statistically the same, i.e. the dice is fair
# 2) choose the test: chi2 test to compare categorical data
#         -- the degree of freedom in this case is (6 - 1) * (2 - 1) = 5, but the 
#            routine called below computes that without you having to specify it in this instance
# 3) set the significance: alpha = 0.05
# 4) compte some things

alpha = 0.05

# if f_exp not provided then code assumes uniform distribution (which is actually the case here)
chi2, p_val = stats.chisquare(df["freq"], f_exp=df["expected_freq"])

print("====================")
print("chi2 test evaluation")
print("====================")
print(f"null hypothesis is that dice is fair, threshold value of {alpha:.2f}")
print(f"(real answer here in this case is that it is fair)")
print(" ")
print(f"p-value from data sample is {p_val:.7f}")
print()
if p_val < alpha:
    print(f"  p-value smaller than threshold, REJECT null hypothesis")
    print(" ")
    print(f"    *** dice is not fair, likely at the alpha = {alpha} significance ***")
    print(" ")
else:
    print(f"  p-value not smaller than threshold, FAIL TO REJECT null hypothesis")
    print(" ")
    print(f"    *** not enough evidence to say dice is not fair ***")
    print(" ")
    print(f"(consider sample size, threshold etc.)")

In [ ]:
# introduce bias to make the dice really unfair

side = 6                         # choose number of sides a dice has
n = 100                          # throw n times
rolls = np.zeros(n, dtype=int)   # force output to be integers
for i in range(n):
    rolls[i] = throw_n_sided_dice(side=side, bias=0.5)  # keep bias =< 1
print(f"actual output of rolls = {rolls}")

freq = np.zeros(side)
for i in range(side):
    freq[i] = np.sum(rolls == i+1)          # Q. why the +1s etc?
roll_val = np.arange(side) + 1
expected_freq = 1/side * np.ones(side) * n  # Q. what is this doing?

# write this out into a pandas data frame for ease of reading
print()
print(f"=================================================")
print(f"summary table for the statistics of throw of dice")
print(f"=================================================")
df = pd.DataFrame(columns = ["roll_val", "freq", "expected_freq"])
df["roll_val"], df["freq"], df["expected_freq"] = roll_val, freq, expected_freq
print(df)

alpha = 0.05
chi2, p_val = stats.chisquare(df["freq"], f_exp=df["expected_freq"])

print()
print("====================")
print("chi2 test evaluation")
print("====================")
print(f"null hypothesis is that dice is fair, threshold value of {alpha:.2f}")
print(f"(real answer here in the dice is biased low)")
print(" ")
print(f"two-sided p-value from data sample is {p_val:.7f}")
print()
if p_val < alpha:
    print(f"  p-value smaller than threshold, REJECT null hypothesis")
    print(" ")
    print(f"    *** dice is not fair, likely at the alpha = {alpha} significance ***")
    print(" ")
else:
    print(f"  p-value not smaller than threshold, FAIL TO REJECT null hypothesis")
    print(" ")
    print(f"    *** not enough evidence to say dice is not fair ***")
    print(" ")
    print(f"(consider sample size, threshold etc.)")
    
# Q. plot out the historgram as well, as well as the histogram that you actually expect
# Q. try turning down the bias but keeping the number of throws the same, when do you start not being
#    able to reliably reject the null hypothesis?
# Q. keeping a low-ish bias, investigate the behaviour with increasing number of throws to start reliably
#    rejecting the null hypothesis
# Q. (harder) compute numerically the Type II error (see 05) in the (bias, number of throws) parameter space
#             -- probably display this as a heatmap or a contour map of sorts (Google this, or see e.g. 09)
#             -- think about what the answer you expect before you go ahead and do it
#             -- for each choice of (bias, number of throws), you probably want to do no less than 10 experiments
#                to get some sort of statistical significance (otherwise you graph will probably change
#                noticeably every time you run the code)

-----------------
# c) revisiting iris data: ANOVA and $F$-tests

In [ ]:
# revisiting iris data: similar statistical test, and ANOVA + F-tests
# I'm going to quickly fly through the iris data set and do a t-test and chi2 test on these
# (see 04 and 05 as well)

# Q. convince yourself I did the below stuff right, or, if not, point out the error 
#    (coding, statistical and or scientific)
#    -- add some comments in for example (I have delibrately not added too many comments below)
#    -- hint: there are a non-zero amount of these, I am doing this mostly to demonstrate syntax)
#
# I am almost redoing some of the following R stuff but in python
# -- https://svaditya.github.io/oldblog/chi_square_and_t_tests_on_iris_data.html

df = pd.read_csv("iris.csv")
setosa_data = df.loc[df["variety"] == "Setosa"]
versic_data = df.loc[df["variety"] == "Versicolor"]
virgin_data = df.loc[df["variety"] == "Virginica"]

In [ ]:
# t-test: want to show PETAL.WIDTH is affected by SPECIES

# box-and-whisker plot to see the data
fig = plt.figure(figsize=(4, 4))
ax = plt.axes()
ax.boxplot([setosa_data["petal.length"], versic_data["petal.width"]])  # !!!!! calling wrong variables
ax.set_xticklabels(["setosa", "versicolor"])
ax.set_xlabel(r"petal.width")
ax.set_ylabel(r"length (m)")
ax.grid()

alpha = 0.05  # standard number from Fisher

t_val, p_val = stats.ttest_rel(setosa_data["petal.length"], versic_data["petal.width"])  # !!! number of entries!

print("=================")
print("t test evaluation")
print("=================")
print(f"null hypothesis of WHAT?, threshold value of {alpha:.2f}")
print(" ")
print(f"two-sided p-value from data sample is {p_val:.7f}")
print()
if p_val < alpha:
    print(f"  p-value smaller than threshold, REJECT null hypothesis")
    print(" ")
    print(f"    *** CONCLUSION? likely at the alpha = {alpha} significance ***")
    print(" ")
else:
    print(f"  p-value not smaller than threshold, ACCEPT the null hypothesis")  # !!!!!
    print(" ")
    print(f"    *** CONCLUSION??? ***")
    print(" ")

In [ ]:
# chi2-test: want to show PETAL.LENGTH is affected by SPECIES

# box-and-whisker plot to see the data
fig = plt.figure(figsize=(4, 4))
ax = plt.axes()
ax.boxplot([setosa_data["petal.length"], versic_data["petal.width"]])  # !!!!! calling wrong variables
ax.set_xticklabels(["setosa", "versicolor"])
ax.set_xlabel(r"petal.width")
ax.set_ylabel(r"length (m)")
ax.grid()

# Q. what is this doing? and why are we doing it?
mean_length = np.mean(np.concatenate([virgin_data["petal.length"].values, versic_data["petal.length"].values]))
df = pd.DataFrame(columns = ["above_mean", "virginica", "versicolor"])
df["above_mean"] = ["yes", "no"]
df["virginica"] = [np.sum(virgin_data["petal.length"] > mean_length), 
                   np.sum(virgin_data["petal.length"] < mean_length)]
df["versicolor"] = [np.sum(versic_data["sepal.length"] > mean_length),  # !!!
                    np.sum(versic_data["sepal.length"] < mean_length)]

print(df)

chi2, p_val = stats.chisquare(df["virginica"], f_exp=df["versicolor"])  # !!! scientific issue

print()
print("====================")
print("chi2 test evaluation")
print("====================")
print(f"null hypothesis is WHAT, threshold value of {alpha:.2f}")
print(" ")
print(f"p-value from data sample is {p_val:.7f}")
print()
if p_val < alpha:
    print(f"  p-value smaller than threshold, REJECT null hypothesis")
    print(" ")
    print(f"    *** WHAT? likely at the alpha = {alpha} significance ***")
    print(" ")
else:
    print(f"  p-value not smaller than threshold, ACCEPT null hypothesis") # !!!
    print(" ")
    print(f"    *** WHAT? ***")
    print(" ")

## ANOVA and $F$-tests

In [ ]:
# ANOVA
#
# the tests so far only allow you to compare two things, but suppose you want to compare the whole group
# e.g. for Iris data, are the means of whatever attribute between the three groups the same, or different?
# lets compute these first to see anyway

df = pd.read_csv("iris.csv")
setosa_data = df.loc[df["variety"] == "Setosa"]
versic_data = df.loc[df["variety"] == "Versicolor"]
virgin_data = df.loc[df["variety"] == "Virginica"]

df_summary = pd.DataFrame(columns = ["stat", "Setosa", "Versicolor", "Virginica", "Total"])
df_summary["stat"] = ["sepal.length.mean", "sepal.width.mean", 
                      "petal.length.mean", "petal.width.mean"]
df_summary["Setosa"]     = setosa_data.describe().values[1]  # 2nd index is the mean
df_summary["Versicolor"] = versic_data.describe().values[1]
df_summary["Virginica"]  = virgin_data.describe().values[1]
df_summary["Total"]      = df.describe().values[1]
df_summary

# the means are clearly different, but are they different enough?

In [ ]:
# ANOVA test
#
# the point here is you don't want to run multiple pairs of tests (e.g. with a,b,c, a vs. b, a vs. c, b vs. c etc.)
# because you are then more likely to run into Type I errors of false positives
#
# ultimately you want to try and identify a variable that is able to distinguish the data you see, and here the
# hypothesis is that the species of flower is a good predictor for the various attributes (although we are 
# only going to do one here)
#
# 1) null hypothesis: the means between all species are all the same
#    -- alternative hypothesis: at least a pair is different 
#                               (this is what we should be getting with the Iris data probably)
# 2) decide on the test: going to compare one variable over three groups, so ANOVA one way F-test 
# 3) significance: alpha = 0.05
# 4) compute some stuff
#    -- could do this manually, or just do it with package
#    -- p-val is usual thing
#    -- f-val is the variance and ratio within and between groups (close to 1 = not much difference)

alpha = 0.5
var = "sepal.width"
f_val, p_val = stats.f_oneway(setosa_data[var], versic_data[var], virgin_data[var])

print("=================")
print("F-test evaluation")
print("=================")
print(f"null hypothesis that all means are the same, threshold value of {alpha:.2f}")
print(f"(for the Iris data the means between Setosa and the rest are quite different)")
print(" ")
print(f"one-sided f-value from data set is {f_val:.7f}, p-value is {p_val:.7f}")
print()
if p_val < alpha:
    print(f"  p-value smaller than threshold, REJECT null hypothesis")
    print(" ")
    print(f"    *** at least one pair with different mean, likely at the alpha = {alpha} significance ***")
    print(" ")
else:
    print(f"  p-value not smaller than threshold, FAIL TO REJECT the null hypothesis")
    print(" ")
    print(f"    *** cannot conclude means are different ***")
    print(" ")

# high f-val here is ratio of variance of sample groups and variance in sample groups
# i.e. large f-val means variation between groups dominate the variance within the groups

# Q. try it for other variables too
# Q. the analyses is for one variable and its dependence on categorical variables, look up and see if you can
#    do multiple variables and its dependence on categorical variables

In [ ]:
# notes:
# 1) F-test tells you the means are different but it doesn't tell you which pair is different (see Tukey HSD later)
# 2) F-test is sensitive to whether the data are independent, normally distributed and homogeneous variances etc.
#
# I have skipped some extra tests here (saved here somewhat by CLT), but you can (and should) 
# run some pre- and post-tests to check various things
#   -- plot out the histogram?
#   -- Shapiro-Wilk's test ("scipy.stats.shapiro"),  for normality
#   -- Levene's       test ("scipy.stats.levene"),   for homogenity of variances
#   -- Bartlett's     test ("scipy.stats.bartlett"), for homogenity of variances
#   -- Tukey's honest significance test, which is in the "statsmodel" package
#      e.g. from statsmodels.stats.multicomp import pairwise_tukeyhsd
#
# Q. try a few of these tests named above
#    -- look up Google for syntax
# Q. (tedious?) try and do the same as what I have done here but using the "statsmodel" package

In [ ]:
# exercises:
# 1) try a few with some other data you find (or you could try the assignment set of data)
# 2) (hard-ish) similar to the A/BIC introduced in 04, the f-val can be used to say whether a linear regression 
#               model is significantly better than another, weighted accordingly by the number of 
#               parameters (look up "F-test" on Wikipedia, towards the end of the page)
#               try and adapt the existing F-test commands etc. to test for goodness of fit between two
#               models
#               -- stats.f_oneway allows for comparison of two arrays

In [ ]:
# conceptual exercise of sorts:
#
# Around Chinese New Year I was talking with my family (alcohol may or may not have been involved), and I noted
# that Miffy seems to have a tendency to poo around the time when we are eating. This might of course be bias, but
# it is not inconeivable she is doing this out of spite.
#   
# The conversation might have got on to how might one actually try and ascertain this (again, alcohol may or may
# not have been involved)
#
# How might you actually test this statistically?
# What kind of statistical power might you want?
# Deciding on the choice of power, how many samples might you need?
# Suppose the frequency really was SOMETHING, carry out your proposed statistical test in Python and conclude
#   things accordingly (what assumptions do you need?)
# There are of course confounding factors, since the time might happen to be correlated. Maybe modify your
#   assumption above and repeat the tests accordingly

<img src="https://c.tenor.com/NAH83oVLQLQAAAAC/movie-action.gif" width="400" alt='inigo'>

(Random reference regarding the word "inconceivable"; fake internet points for knowing the pop culture reference)